# Initial setting 

## DeviceSetup

In [1]:
from laboneq.simple import SHFQC
from laboneq.dsl.device import DeviceSetup, create_connection
import numpy as np
from copy import deepcopy

host = "10.172.11.217"
#instantiate ad device setup object
setup = DeviceSetup(uid="test_setup")
#add a dataserver
setup.add_dataserver(host=host, port="8004",uid="test_server")

setup.add_instruments(SHFQC(uid="test_shfqc",server_uid="test_server", address="DEV12576", device_options="SHFQC/PLUS/QC6CH"))

setup.instruments

[SHFQC(
│   uid='test_shfqc',
│   interface='1GbE',
│   connections=[],
│   server_uid='test_server',
│   address='DEV12576',
│   device_options='SHFQC/PLUS/QC6CH',
│   reference_clock_source=None,
│   device_class=0
)
]

In [2]:
for ch_idx, q_idx in enumerate([15,16,17,18,19]):# enumerate 안에 len(list) 강제하고싶음
    #QA
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/measure", ports="QACHANNELS/0/OUTPUT")) #signal_type="iq"
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/acquire", ports="QACHANNELS/0/INPUT" )) #signal_type="acquire"
    #SG
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_ef", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_cr", ports=f"SGCHANNELS/{ch_idx}/OUTPUT"))#signal_type="iq"


## Setting QuantumElement, QPU and QuantumPlaform(optional)
1. By using `from_device_setup` function  
    DeviceSetup object we made doesn't have Calibration
2. By using `load_qubit_parameters` function
    This way 
- load parameters from JSON

In [3]:
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
qubits = list(_loaded_qubits.values())
qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


## Common parameters check

In [4]:
for _, qubit in enumerate(qubits):
    ##################LOCAL OSCILLATOR FREQUENCY AND CHANNEL POWER ###################################
    qubit.parameters.drive_lo_frequency = 4.8e9
    qubit.parameters.readout_lo_frequency = 7.0e9
    qubit.parameters.readout_range_in =  0
    qubit.parameters.readout_range_out = -25
    qubit.parameters.drive_range = 0 
    
    
    
    ############READOUT PARAMETERS###################################################################
    qubit.parameters.readout_length - 2.0e-6
    qubit.parameters.readout_pulse['function'] = 'GaussianSquare'
    qubit.parameters.readout_pulse['sigma'] = 0.1
    qubit.parameters.readout_pulse['width'] = None
    qubit.parameters.readout_pulse['risefall_sigma_ratio'] = 3.0
    qubit.parameters.readout_amplitude = 1.0  #WARNING! 이거 SWEEP 할때 오버라이딩이 안되고 여기서 SCALING 됨
    
    qubit.parameters.reset_delay_length = 200e-6
    ###########SPECTROSCOPY PARAMETERS###############################################################
    # Pulsed qubit spectroscopy drive ,  
    qubit.parameters.spectroscopy_pulse['function'] = 'GaussianSquare'
    qubit.parameters.spectroscopy_length = 5.0e-6
    qubit.parameters.spectroscopy_pulse['sigma'] = 0.2
    qubit.parameters.spectroscopy_pulse['width'] = None
    qubit.parameters.spectroscopy_pulse['risefall_sigma_ratio'] = 3.0
    qubit.parameters.spectroscopy_amplitude=0.01 
    
    
    ###########TRANSITION PARAMETERS#####################################
    qubit.parameters.ge_drive_amplitude_pi = 1.0
    qubit.parameters.ge_drive_amplitude_pi2 = 0.5
    qubit.parameters.ef_drive_amplitude_pi = 1.0
    qubit.parameters.ef_drive_amplitude_pi2 = 0.5 # Initialization just to avoid potential scaling problem
    

qubits[3].parameters.readout_amplitude = 0.5
qubits[4].parameters.readout_amplitude = 0.5
qubits[2].parameters.readout_amplitude = 0.5


In [5]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True)

[2025.08.18 17:03:55.362] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /home/kriss/JSAHN/qubit-experiment/laboneq_output/log
[2025.08.18 17:03:55.363] INFO    VERSION: laboneq 2.57.0
[2025.08.18 17:03:55.364] INFO    Connecting to data server at 10.172.11.217:8004
[2025.08.18 17:03:55.370] INFO    Connected to Zurich Instruments LabOne Data Server version 25.04.1.17 at 10.172.11.217:8004
[2025.08.18 17:03:55.371] WARNING Ignoring that Version of LabOne Data Server (25.04.1.17) and Python API (25.07.0.507) do not match.
[2025.08.18 17:03:55.381] INFO    Configuring the device setup
[2025.08.18 17:03:55.388] INFO    The device setup is configured


# 1 Spectroscopy

Problem on options
If option is given as `experiment_workflow.options()` we cannot change options for create_experiment.
To change options for experiment we have to give `ResonatorSpectroscopyExperimentOptions'

## 1.1 Resonator Spectroscopy
- Default `AcquisitioType.SPECTROSOCPY_IQ`

In [ ]:
from laboneq.simple import AcquisitionType
from laboneq_applications.experiments import resonator_spectroscopy
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.readout_amplitude = 0.1
temporary_parameters.readout_length = 2e-6
###############################################################
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-5e6,+5e6,101)
################################################################
options = resonator_spectroscopy.experiment_workflow.options()
options.update(True)
#print(workflow.show_fields(options))

###################################################################
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options,
    temporary_parameters={q.uid : temporary_parameters}
)
#res_spec_compiled = session.compile(resonator_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies,options=options))
res_spec_result = res_spec.run()
print(res_spec_result.tasks['analysis_workflow'].output)


## 1.1 + Resonator Spectroscopy Amplitude

In [ ]:
from laboneq.simple import AcquisitionType
from laboneq_applications.experiments import resonator_spectroscopy_amplitude
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.readout_amplitude = 1.0
temporary_parameters.readout_length = 2e-6
###############################################################
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-2e6,+2e6,101)
amplitudes = np.linspace(0,1.0,11)
################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#print(workflow.show_fields(options))

###################################################################
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options,
    temporary_parameters={q.uid : temporary_parameters}
)
#res_spec_amp_compiled = session.compile(resonator_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies,options=options))
res_spec_amp_result = res_spec_amp.run()


## 1.2 Qubit Spectroscopy

In [ ]:
from copy import deepcopy
from laboneq_applications.experiments import qubit_spectroscopy
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.spectroscopy_amplitude = 0.001
temporary_parameters.spectroscopy_length = 30e-6
###############################################################
frequencies = q.parameters.resonance_frequency_ge +np.linspace(-2e6,2e6,201)
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(True)

#print(workflow.show_fields(options))

###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

qubit_spec_result = qubit_spec.run()
#qubit_spec_compiled = session.compile(qubit_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies, options=options))
qubit_parameters = qubit_spec_result.tasks['analysis_workflow'].output
print(qubit_parameters)


## 1.2 + Qubit Spectroscopy Amplitude

In [ ]:
from copy import deepcopy
from laboneq_applications.experiments import qubit_spectroscopy_amplitude
q = qubits[3]
temporary_parameters = q.parameters.copy()
#temporary_parameters.spectroscopy_amplitude = 0.01
#temporary_parameters.spectroscopy_length
###############################################################
frequencies = q.parameters.resonance_frequency_ge +np.linspace(-5e6,5e6,11)
amplitudes = np.linspace(0.5, 1.0,11)
################################################################
options = qubit_spectroscopy_amplitude.experiment_workflow.options()

#print(workflow.show_fields(options))

###################################################################
qubit_spec_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

qubit_spec_amp_result = qubit_spec_amp.run()
#qubit_spec_compiled = session.compile(qubit_spectroscopy_amplitude.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies,amplitudes=amplitudes, options=options))
print(qubit_spec_amp_result.tasks['analysis_workflow'].output)

# 2. Rabi Amplitude

In [ ]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.ge_drive_amplitude_pi=1.0
#######################################################################
amplitudes = np.linspace(0,1.0,51)
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(True)
options.use_cal_traces(False)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_result.tasks['analysis_workflow'].output)

# 3. Ramsey

# WARNING 
현재 자동으로 q17_readout_acquire_osc' on signal '/logical_signal_groups/q17/acquire' to SOFTWARE 로 설정되는거 어디서 하는지 확인해야함 , cr_Drive도 AUTO인데 뭐지.....

하여튼 HARDWARE여야함

In [ ]:
from laboneq_applications.experiments import ramsey
q = qubits[3]
temporary_parameters = q.parameters.copy()
#######################################################################
delays = np.linspace(0,1e-6,10)
detunings = 0.2e6
#######################################################################
# options = ramsey.experiment_workflow.options()
# options.transition("ge")
# options.cal_states("ge")
# options.update(True)
#print(workflow.show_fields(options))

###################################################################
ramsey_freq = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    #temporary_parameters={q.uid : temporary_parameters},
    #options=options
)

ramsey_freq_result = ramsey_freq.run()
#ramsey_compiled = session.compile(ramsey.create_experiment(qpu=qpu,qubit=q,delays=delays,detunings=detunings, options=options))
print(ramsey_freq_result.tasks['analysis_workflow'].output)

# Error Amplification

In [ ]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(True)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

In [ ]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(True)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
error_amp_half = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_half_result = error_amp_half.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_half_result.tasks['analysis_workflow'].output)

In [ ]:
from laboneq_applications.experiments import drag_q_scaling

# Dispersive shift

In [ ]:
from laboneq_applications.experiments import dispersive_shift
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
frequencies=q.parameters.readout_resonator_frequency + np.linspace(-5e6,+5e6,101)
states=["g","e"]

#######################################################################
options = dispersive_shift.experiment_workflow.options()
options.update(True)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
dispersive = dispersive_shift.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    states=states,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

dispersive_result = dispersive.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(dispersive_result.tasks['analysis_workflow'].output)

# IQ Blobs

In [ ]:
from laboneq_applications.experiments import iq_blobs
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################

states=["g","e"]

#######################################################################
options = iq_blobs.experiment_workflow.options()
#options.update(False)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
iq = iq_blobs.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    states=states,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

iq_result = iq.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))


In [ ]:
from laboneq_applications.experiments import time_traces

# Save qubit

In [ ]:
# from helper import load_qubit_parameters, save_qubit_parameters
# save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="kaist")

In [19]:
from laboneq_applications.tasks.parameter_updating import (
    temporary_qpu,
    temporary_quantum_elements_from_qpu,
    update_qubits,
)
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.spectroscopy_amplitude = 0.001
temporary_parameters.spectroscopy_length = 30e-6
temp_qpu = temporary_qpu(qpu, {q.uid : temporary_parameters})

qubits = temporary_quantum_elements_from_qpu(temp_qpu, qubits)

qubits[3].spectroscopy_parameters()

('drive',
 {'amplitude': 0.001,
  'length': 3e-05,
  'pulse': {'function': 'GaussianSquare',
   'sigma': 0.2,
   'width': None,
   'risefall_sigma_ratio': 3.0}})

# f state

In [25]:
from experiments import qubit_spectroscopy
from copy import deepcopy

q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.spectroscopy_amplitude = 0.01
temporary_parameters.spectroscopy_length = 2e-6
###############################################################
frequencies = q.parameters.resonance_frequency_ge+np.linspace(-300e6 , 0e6,501)  #4.3685e
################################################################

options = qubit_spectroscopy.experiment_workflow.options()
options.update(False)

#print(workflow.show_fields(options))

###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

qubit_spec_result = qubit_spec.run()

qubit_spec_compiled = session.compile(qubit_spectroscopy.create_experiment(qpu=qpu,qubits=q,frequencies=frequencies))
#qubit_parameters = qubit_spec_result.tasks['analysis_workflow'].output
#print(qubit_parameters)








[2025.08.18 17:40:26.182] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.18 17:40:26.183] INFO      Workflow 'qubit_spectroscopy': execution started at 2025-08-18                
[2025.08.18 17:40:26.183] INFO      08:40:26.182159Z                                                              
[2025.08.18 17:40:26.183] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.18 17:40:26.186] WARNING Type <class 'dict'> not supported by the serializer [name: qubit_spectroscopy.input.temporary_parameters].
[2025.08.18 17:40:26.193] INFO    Task 'temporary_qpu': started at 2025-08-18 08:40:26.193437Z
[2025.08.18 17:40:26.194] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.18 17:40:26.194] INFO    Task 'temporary_qpu': ended at 2025-08-18 08:40:26.194646Z
[2025.08.18 17:40:26.197] INFO    Task 'temporary_quantum_elements_from_qpu

In [21]:

from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
show_pulse_sheet("first_comp", qubit_spec_compiled, interactive=True)
#plot_simulation(qubit_spec_compiled, length=2e-6)




[2025.08.18 17:25:59.599] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2025.08.18 17:25:59.601] INFO    Resolved modulation type of oscillator 'q18_readout_acquire_osc' on signal '/logical_signal_groups/q18/acquire' to SOFTWARE
[2025.08.18 17:25:59.602] INFO    Resolved modulation type of oscillator 'q18_drive_ge_osc' on signal '/logical_signal_groups/q18/drive' to HARDWARE
[2025.08.18 17:25:59.602] INFO    Resolved modulation type of oscillator 'q18_drive_ef_osc' on signal '/logical_signal_groups/q18/drive_ef' to HARDWARE
[2025.08.18 17:25:59.602] INFO    Starting LabOne Q Compiler run...
[2025.08.18 17:25:59.636] INFO    Schedule completed. [0.033 s]
[2025.08.18 17:25:59.655] INFO    Code generation completed for all AWGs. [0.019 s]
[2025.08.18 17:25:59.656] INFO    Completed compilation step 1 of 1. [0